<a href="https://colab.research.google.com/github/PRAISE-KING/wk-6-ai-for-software-engineering/blob/main/Edge_AI_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# import necessary resouces

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

import pandas as pd


In [6]:

# Step 2: Load CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Flatten labels
y_train = y_train.flatten()
y_test = y_test.flatten()

# Filter for class 0 (airplane) and class 1 (automobile)
train_filter = (y_train == 0) | (y_train == 1)
test_filter = (y_test == 0) | (y_test == 1)

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

# Normalize and resize images to 96x96
x_train = tf.image.resize(x_train / 255.0, [96, 96])
x_test = tf.image.resize(x_test / 255.0, [96, 96])

# One-hot encode the labels (2 classes)
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)


In [7]:
# Step 3: Build Model
base = MobileNetV2(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
x = GlobalAveragePooling2D()(base.output)
out = Dense(2, activation='softmax')(x)
model = Model(base.input, out)

for layer in base.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 261ms/step - accuracy: 0.9172 - loss: 0.1897 - val_accuracy: 0.9765 - val_loss: 0.0677
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 145s 272ms/step - accuracy: 0.9787 - loss: 0.0615 - val_accuracy: 0.9785 - val_loss: 0.0604
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 142s 272ms/step - accuracy: 0.9823 - loss: 0.0495 - val_accuracy: 0.9770 - val_loss: 0.0622


In [8]:
# Step 4: Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpcwn2pefp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  139142336995280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336995472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336996432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336997584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336996816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336995856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336997008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336997968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336998352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336995088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139142336998736:

In [9]:
# Save TFLite model
with open("edge_model.tflite", "wb") as f:
    f.write(tflite_model)

In [11]:
# Load TFLite model
import numpy as np

interpreter = tf.lite.Interpreter(model_path="edge_model.tflite")
interpreter.allocate_tensors()

# Get input/output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run inference on a test image
test_img = np.expand_dims(x_test[0], axis=0).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], test_img)
interpreter.invoke()
prediction = interpreter.get_tensor(output_details[0]['index'])

print("Predicted Class:", "Airplane" if np.argmax(prediction) == 0 else "Automobile")

Predicted Class: Airplane
